关联分析：
计算支持度，包含该项的记录数/总记录数
计算可信度，「尿布」-〉「啤酒」可信度=「尿布，啤酒」支持度/「尿布」支持度

In [83]:
import copy
#辅助函数：生成基础候选项集C1
def createC1(dataset):
    C1 = [] #C1 是大小为1的所有候选项集的集合
    for transaction in dataset:
        for item in transaction:
            if not [item] in C1:
                C1.append([item])
    C1.sort()
    print ("C1=",C1)
    return map(frozenset,C1)#不可改变的set元素集合


#辅助函数：计算候选项集的每项支持度，去除低于最小支持度的项
def scanD(D,Ck,minSupport): #D数据集，候选集Ck，最小支持度minSupport
    ssCnt={}
    
    for tid in D:
        #print (tid)
        C = copy.deepcopy(Ck)#ck的类型是map，map中的对象只能遍历一次，所以此处用deepcopy来每次获得不同地址的ck，才能遍历多次
        for can in C:
            #print (can)
            if can.issubset(tid):
                #print ("yes")
                if can not in ssCnt: ssCnt[can]=1
                else: ssCnt[can]+=1 #候选集的元素在数据集交易中出现的次数
    print (ssCnt)               
    numItems = float(len(D))
    retList = []
    supportData = {}
    for key in ssCnt:
        support = ssCnt[key]/numItems #计算候选集项的支持度，包含该项的记录数/总记录数
        if support >= minSupport:
            retList.insert(0,key)#大于最小支持度的项存储到retList新的候选项集（在列表首部插入新的项），和supportData新候选项集的支持度
            supportData[key] = support
    return retList,supportData

In [84]:
dataset = [[1,3,4],[2,3,5],[1,2,3,5],[2,5]]
C1 = createC1(dataset)
print (C1)

C1= [[1], [2], [3], [4], [5]]


In [85]:

L1,supportData1 = scanD(dataset,C1,0.5)

{frozenset({1}): 2, frozenset({3}): 3, frozenset({4}): 1, frozenset({2}): 3, frozenset({5}): 3}


In [86]:
L1

[frozenset({5}), frozenset({2}), frozenset({3}), frozenset({1})]

In [87]:
supportData1

{frozenset({1}): 0.5,
 frozenset({3}): 0.75,
 frozenset({2}): 0.75,
 frozenset({5}): 0.75}

In [88]:
#Aprioir算法
def aprioriGen(Lk,k):#生成Ck（组合后的频繁项集）,Lk为输入频繁项集，k是组合生成的频繁项集每一项的元素个数
    retList= []
    lenLk= len(Lk)
    for i in range(lenLk):
        for j in range(i+1,lenLk):
            L1= list(Lk[i])[:k-2]; L2 = list(Lk[j])[:k-2]
            L1.sort();L2.sort()
            if L1==L2:
                retList.append(Lk[i]|Lk[j])#将「0，1」「0，2」合并成「0，1，2」 ｜取或
    return retList

def apriori(dataset, minSupport=0.5):
    C1 = createC1(dataset)
    #D = map(set,dataset)
    L1,supportData = scanD(dataset,C1,minSupport)#从频繁集C1（单元素）开始
    L = [L1]
    k = 2#k至少为2
    while (len(L[k-2])>0): 
        Ck = aprioirGen(L[k-2],k) #将频繁集每次向前扩展一个元素，计算扩展后的频繁集的项支持度
        Lk,supK = scanD(dataset,Ck,minSupport)
        supportData.update(supK)
        L.append(Lk)
        k+=1
    return L, supportData

In [89]:
L,suppData = apriori(dataset)
L

C1= [[1], [2], [3], [4], [5]]
{frozenset({1}): 2, frozenset({3}): 3, frozenset({4}): 1, frozenset({2}): 3, frozenset({5}): 3}
{frozenset({1, 3}): 2, frozenset({2, 5}): 3, frozenset({3, 5}): 2, frozenset({2, 3}): 2, frozenset({1, 5}): 1, frozenset({1, 2}): 1}
{frozenset({2, 3, 5}): 2}
{}


[[frozenset({5}), frozenset({2}), frozenset({3}), frozenset({1})],
 [frozenset({2, 3}), frozenset({3, 5}), frozenset({2, 5}), frozenset({1, 3})],
 [frozenset({2, 3, 5})],
 []]

In [90]:
suppData

{frozenset({1}): 0.5,
 frozenset({3}): 0.75,
 frozenset({2}): 0.75,
 frozenset({5}): 0.75,
 frozenset({1, 3}): 0.5,
 frozenset({2, 5}): 0.75,
 frozenset({3, 5}): 0.5,
 frozenset({2, 3}): 0.5,
 frozenset({2, 3, 5}): 0.5}

In [114]:
#关联规则生成函数
def generateRules(L,supportData,minConf = 0.7):#频繁项集L，支持度字典supportData，最下可信度minConf
    bigRuleList =[]
    for i in range(1,len(L)):
        
        for freqset in L[i]:#对于频繁项集中的每一项
            H1 = [frozenset([item]) for item in freqset]
            if(i>1):#L2每项才有3个元素
                rulesFromConseq(freqset,H1,supportData,bigRuleList,minConf)
            else:#L1每项只有两个元素,i=1开始
                #print ("eeee",H1)
                calcConf(freqset,H1,supportData,bigRuleList,minConf)
    return bigRuleList

#计算可信度，「尿布」-〉「啤酒」可信度=「尿布，啤酒」支持度/「尿布」支持度
def calcConf(freqset,H,supportData,br1,minConf=0.7):#H是可以出现在规则右部的元素列表
    prunedH = []
    for conseq in H:#遍历H的所有项集来计算他们的可信度
        conf= supportData[freqset]/supportData[freqset-conseq]
        if conf>=minConf:
            print (freqset-conseq,'--->',conseq,'conf:',conf)
            br1.append((freqset-conseq,conseq,conf))
            prunedH.append(conseq)
            
    return prunedH#可信度大于最小可信的的项

#扩展规则右部元素的大小，计算可信度
def rulesFromConseq(freqset,H,supportData,br1,minConf=0.7):
    m = len(H[0])#规则右部的元素大小m
    if(len(freqset)>(m+1)):#频繁项是否大到可以移除大小为m的子集
        print (m,H)
        Hmp1 = aprioriGen(H,m+1)#将规则右部元素扩展一个元素
        print ("eeeee",Hmp1)
        Hmp1 = calcConf(freqset,Hmp1,supportData,br1,minConf)#计算规则右部元素的可信度
        if (len(Hmp1)>1):#有多个后续H（prunedH）
            
            #print ("0ffff",Hmp1)
            rulesFromConseq(freqset,Hmp1,supportData,br1,minConf)
    
            

In [115]:

L,suppData = apriori(dataset)
rules = generateRules(L,suppData,minConf=0.7)

C1= [[1], [2], [3], [4], [5]]
{frozenset({1}): 2, frozenset({3}): 3, frozenset({4}): 1, frozenset({2}): 3, frozenset({5}): 3}
{frozenset({1, 3}): 2, frozenset({2, 5}): 3, frozenset({3, 5}): 2, frozenset({2, 3}): 2, frozenset({1, 5}): 1, frozenset({1, 2}): 1}
{frozenset({2, 3, 5}): 2}
{}
frozenset({5}) ---> frozenset({2}) conf: 1.0
frozenset({2}) ---> frozenset({5}) conf: 1.0
frozenset({1}) ---> frozenset({3}) conf: 1.0
1 [frozenset({2}), frozenset({3}), frozenset({5})]
eeeee [frozenset({2, 3}), frozenset({2, 5}), frozenset({3, 5})]


In [116]:
L,suppData = apriori(dataset)
rules = generateRules(L,suppData,minConf=0.5)

C1= [[1], [2], [3], [4], [5]]
{frozenset({1}): 2, frozenset({3}): 3, frozenset({4}): 1, frozenset({2}): 3, frozenset({5}): 3}
{frozenset({1, 3}): 2, frozenset({2, 5}): 3, frozenset({3, 5}): 2, frozenset({2, 3}): 2, frozenset({1, 5}): 1, frozenset({1, 2}): 1}
{frozenset({2, 3, 5}): 2}
{}
frozenset({3}) ---> frozenset({2}) conf: 0.6666666666666666
frozenset({2}) ---> frozenset({3}) conf: 0.6666666666666666
frozenset({5}) ---> frozenset({3}) conf: 0.6666666666666666
frozenset({3}) ---> frozenset({5}) conf: 0.6666666666666666
frozenset({5}) ---> frozenset({2}) conf: 1.0
frozenset({2}) ---> frozenset({5}) conf: 1.0
frozenset({3}) ---> frozenset({1}) conf: 0.6666666666666666
frozenset({1}) ---> frozenset({3}) conf: 1.0
1 [frozenset({2}), frozenset({3}), frozenset({5})]
eeeee [frozenset({2, 3}), frozenset({2, 5}), frozenset({3, 5})]
frozenset({5}) ---> frozenset({2, 3}) conf: 0.6666666666666666
frozenset({3}) ---> frozenset({2, 5}) conf: 0.6666666666666666
frozenset({2}) ---> frozenset({3, 5}

In [117]:
rules

[(frozenset({3}), frozenset({2}), 0.6666666666666666),
 (frozenset({2}), frozenset({3}), 0.6666666666666666),
 (frozenset({5}), frozenset({3}), 0.6666666666666666),
 (frozenset({3}), frozenset({5}), 0.6666666666666666),
 (frozenset({5}), frozenset({2}), 1.0),
 (frozenset({2}), frozenset({5}), 1.0),
 (frozenset({3}), frozenset({1}), 0.6666666666666666),
 (frozenset({1}), frozenset({3}), 1.0),
 (frozenset({5}), frozenset({2, 3}), 0.6666666666666666),
 (frozenset({3}), frozenset({2, 5}), 0.6666666666666666),
 (frozenset({2}), frozenset({3, 5}), 0.6666666666666666)]